In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from ecg_dataset import ECG_dataset
from svm_plot import plot_decision_regions
import heartpy as hp


def extract_ecg_features(ecg_signal, sampling_rate):
    # 提取R波峰值
    wd, m = hp.process(ecg_signal, sample_rate=sampling_rate, bpmmin=1, bpmmax=1000)

    # 心率
    hr = m['bpm']

    # 心率变异性 - 时域
    hrv_time = m['sdnn']

    # R波峰值间隔
    rr_intervals = np.diff(wd['peaklist'])

    # 计算波形复杂度 (示例: 标准差)
    waveform_complexity = np.std(ecg_signal)

    return [hr, hrv_time, np.mean(rr_intervals), waveform_complexity]


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

In [2]:
heart_rate_data = ECG_dataset('./')
# d = heart_rate_data[1]
# x = np.linspace(0, 3000, 3000)  # x轴坐标值
# plt.plot(x, d[0], c='r')  # 参数c为color简写，表示颜色,r为red即红色
# plt.show()  # 显示图像'''
sample_rating = 100
features = []
label = []

for i in range(len(heart_rate_data)):
    data = (heart_rate_data[i][0].numpy() - np.min(heart_rate_data[i][0].numpy())) / (np.max(heart_rate_data[i][0].numpy()) - np.min(heart_rate_data[i][0].numpy()))    # 数据归一化

    data = butter_bandpass_filter(data, 0.5, 40, sample_rating) # 滤波
    p = extract_ecg_features(data, sample_rating)   # 提取特征
    features.append(np.array(p))

    label.append(np.array(heart_rate_data[i][1]))

label = np.array(label)
features = np.array(features)
features[np.isnan(features)] = 0
features[np.isinf(features)] = 10000

d:\Anaconda3\envs\proj1\Lib\site-packages\scipy\interpolate\_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
d:\Anaconda3\envs\proj1\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\Anaconda3\envs\proj1\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\Anaconda3\envs\proj1\Lib\site-packages\numpy\ma\core.py:5334: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
d:\Anaconda3\envs\proj1\Lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: 

[[1.98044010e+02 1.64154733e+02 3.45238095e+01 6.95869543e-02]
 [2.36453202e+02 2.23994280e+02 3.20543478e+01 1.03753046e-01]
 [1.89111748e+02 1.45138862e+02 3.48205128e+01 6.71700983e-02]
 ...
 [2.59077527e+02 1.46224348e+02 2.69818182e+01 1.20573188e-01]
 [1.78846154e+02 1.21039549e+02 3.15698925e+01 9.76863718e-02]
 [1.46446518e+02 2.13410476e+02 3.37241379e+01 1.24017011e-01]]


In [15]:
# from sklearn.decomposition import PCA


# pca = PCA(n_components=4)
# pca.fit(features)
# features_new = pca.transform(features)
features_new = features
features_new = features_new - features_new.mean()
features_new = features_new / features_new.std()

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features_new, label, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

svm = SVC()
svm.fit(X_train_scaler, y_train)

pred = svm.predict(X_test_scaler)
print("Accuracy:", accuracy_score(y_test, pred))
print("Classification Report:")
print(classification_report(y_test, pred))
# markers = ('s', 'x', 'o', '^', 'v')
# colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
# for i in range(len(features)):
#     plt.plot(features_new[i, 0], features_new[i, 1], marker=markers[int(label[i])], color=colors[int(label[i])])
# plt.show()
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
clf.fit(X_train_scaler, y_train)
pred = clf.predict(X_test_scaler)
print("Accuracy:", accuracy_score(y_test, pred))
print("Classification Report:")
print(classification_report(y_test, pred))

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train_scaler, y_train)
pred = clf.predict(X_test_scaler)
print("Accuracy:", accuracy_score(y_test, pred))
print("Classification Report:")
print(classification_report(y_test, pred))


d:\Anaconda3\envs\proj1\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9135531135531135
Classification Report:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1247
           1       0.00      0.00      0.00       118

    accuracy                           0.91      1365
   macro avg       0.46      0.50      0.48      1365
weighted avg       0.83      0.91      0.87      1365

Accuracy: 0.9054945054945055
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      1247
           1       0.34      0.10      0.16       118

    accuracy                           0.91      1365
   macro avg       0.63      0.54      0.55      1365
weighted avg       0.87      0.91      0.88      1365



d:\Anaconda3\envs\proj1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda3\envs\proj1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda3\envs\proj1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda3\envs\proj1\Lib\site-packages\sklearn\neighbor

Accuracy: 0.9179487179487179
Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      1247
           1       0.80      0.07      0.12       118

    accuracy                           0.92      1365
   macro avg       0.86      0.53      0.54      1365
weighted avg       0.91      0.92      0.89      1365

